In [11]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

loading the data

In [12]:

def load_data(folder_path, img_size=(48, 48)):
    images, labels = [], []
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for file in os.listdir(label_path):
                file_path = os.path.join(label_path, file)
                img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    images.append(cv2.resize(img, img_size))
                    labels.append(label)
    return images, labels


computing hog features for an image

In [13]:
def compute_hog_features(img, cell_size=(4, 4), block_size=(3, 3), block_stride_factor=2, nbins=12):
    blockStride = (cell_size[0] // block_stride_factor, cell_size[1] // block_stride_factor)
    
    hog = cv2.HOGDescriptor(_winSize=img.shape[:2],
                            _blockSize=(block_size[0] * cell_size[0], block_size[1] * cell_size[1]),
                            _blockStride=blockStride,
                            _cellSize=(cell_size[0], cell_size[1]),
                            _nbins=nbins)
    return hog.compute(img).flatten()



Extracting features from images

In [9]:
def extract_features(images):

    images_uint8 = [np.uint8(img * 255) if img.max() <= 1 else img for img in images]
    return np.array([compute_hog_features(img) for img in images_uint8])


def resize_images(images, target_size=(48, 48)):
    return [cv2.resize(img, target_size, interpolation=cv2.INTER_AREA) for img in images]

def normalize_images(images):
    return [img / 255.0 for img in images]

def apply_clahe(images):
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    return [clahe.apply(img) for img in images]

def denoise_images(images):
    return [cv2.GaussianBlur(img, (5, 5), 0) for img in images]

def correct_illumination(images):
    corrected_images = []
    for img in images:
        img_blur = cv2.GaussianBlur(img, (21, 21), 0)
        img_corrected = cv2.addWeighted(img, 4, img_blur, -4, 128)
        corrected_images.append(img_corrected)
    return corrected_images

def preprocess_images(images):
    images = resize_images(images, target_size=(48, 48))
    images = apply_clahe(images) 
    images = denoise_images(images)
    images = correct_illumination(images)
    images = normalize_images(images)  
    return images

train_path = r"E:\cccc\DATASET2\DATASET\train"
test_path = r"E:\cccc\DATASET2\DATASET\test"

train_images, train_labels = load_data(train_path)
test_images, test_labels = load_data(test_path)

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

train_features = extract_features(train_images)
test_features = extract_features(test_images)


model = SVC(kernel='linear', probability=True)
model.fit(train_features, train_labels_encoded)
predictions = model.predict(test_features)


CALCULATING THE ACCURACY

In [ ]:
accuracy = accuracy_score(test_labels_encoded, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 63.78%
